<a href="https://colab.research.google.com/github/jskaza/nfl-big-data-bowl-2023/blob/master/model_training_multiclass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import datetime
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import math
from sklearn.linear_model import LogisticRegression

2023-01-03 21:01:44.548918: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-03 21:01:45.286759: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-03 21:01:45.286840: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-03 21:01:48.428889: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
input_file_name = 'dataset_week1.json'
filepath = os.path.join(os.getenv('NFL_DATA'), input_file_name)
df = pd.read_json(filepath)

#df = pd.read_json("/content/drive/MyDrive/nfl-big-data-bowl-2023/data/dataset_week1.json")
df.head()

nfl_id                    opponent_x                        opponent_y  \
0   41263  [-2.61, -1.29, 0.1, 1.73, 3]  [0.45, 0.15, -0.54, -0.37, 0.18]   
1   42403  [3, 1.73, 0.1, -1.29, -2.61]  [0.18, -0.37, -0.54, 0.15, 0.45]   
2   44955  [1.73, 0.1, 3, -1.29, -2.61]  [-0.37, -0.54, 0.18, 0.15, 0.45]   
3   53441  [-1.29, 0.1, -2.61, 1.73, 3]  [0.15, -0.54, 0.45, -0.37, 0.18]   
4   53504  [1.73, 3, 0.1, -1.29, -2.61]  [-0.37, 0.18, -0.54, 0.15, 0.45]   

                   teammate_x                    teammate_y  ball_x  ball_y  \
0   [-1.99, 1.26, 2.75, 8.71]  [-2.12, -1.79, -2.14, -2.34]     0.0     0.0   
1  [2.75, 1.26, -1.99, -5.03]  [-2.14, -1.79, -2.12, -1.74]     0.0     0.0   
2  [2.75, -1.99, -5.03, 8.71]  [-2.14, -2.12, -1.74, -2.34]     0.0     0.0   
3   [-5.03, 1.26, 2.75, 8.71]  [-1.74, -1.79, -2.14, -2.34]     0.0     0.0   
4  [1.26, -1.99, 8.71, -5.03]  [-1.79, -2.12, -2.34, -1.74]     0.0     0.0   

      game_id  play_id  frame_id  ...  pff_sack  pff_position_lined_up  \
0  2021090900       97         6  ...         0                    LEO   
1  2021090900       97         6  ...         0                   ROLB   
2  2021090900       97         6  ...         0                    DRT   
3  2021090900       97         6  ...         0                   LILB   
4  2021090900       97         6  ...         0                     RE   

   quarter  down  yards_to_go absolute_yardline_number       personnel_o  \
0        1     3            2                       43  1 RB, 1 TE, 3 WR   
1        1     3            2                       43  1 RB, 1 TE, 3 WR   
2        1     3            2                       43  1 RB, 1 TE, 3 WR   
3        1     3            2                       43  1 RB, 1 TE, 3 WR   
4        1     3            2                       43  1 RB, 1 TE, 3 WR   

   defenders_in_box  offense_formation  score_delta  
0               6.0            SHOTGUN            0  
1               6.0            SHOTGUN            0  
2               6.0            SHOTGUN            0  
3               6.0            SHOTGUN            0  
4               6.0            SHOTGUN            0  

[5 rows x 24 columns]

In [3]:
def make_features(df: pd.DataFrame, feats: list, outcomes: list):
    X, y  = [], []
    grouped_df = df.groupby(["nfl_id","game_id","play_id"])
    for group_name, group_df in grouped_df:
      X.append(group_df[feats].to_numpy())
      y.append(group_df[outcomes].to_numpy()[0])
    
    X = tf.keras.utils.pad_sequences(X ,dtype="float", padding="pre", value= -99)
    y = tf.keras.utils.pad_sequences(y ,dtype="float", padding="pre", value= -99)
    return X, y

In [4]:
feats = ["x","y","ball_x","ball_y"]
outcomes = ["pff_hurry"]
X, y = make_features(df, feats, outcomes)

split = 0.8
num_train = round(split * X.shape[0])

X_train = X[:num_train]
y_train = y[:num_train]

X_test = X[num_train:]
y_test = y[num_train:]


In [5]:
# set random seed for reproducibility

tf.random.set_seed(42)
np.random.seed(42)
NUM_EPOCHS = 10

# Our Model

In [6]:
# create the model
model = tf.keras.Sequential()
model.add(tf.keras.layers.Masking(mask_value=-99.,input_shape= X.shape[1:]))
model.add(tf.keras.layers.LSTM(32, input_shape = X.shape[1:]))
model.add(tf.keras.layers.Dense(y.shape[1], activation="sigmoid"))

# compile and fit the model
model.compile(loss="binary_crossentropy", optimizer="adam", metrics = [tf.keras.metrics.AUC()])

callbacks = [keras.callbacks.EarlyStopping(patience=10, \
    restore_best_weights=True)]

model.fit(
    X_train,
    y_train,
    validation_split=0.15,
    epochs=NUM_EPOCHS,
    #batch_size=64,
    callbacks=callbacks,
)

model.evaluate(X_test, y_test, verbose=1)

2023-01-03 17:14:03.788249: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-01-03 17:14:03.788785: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-01-03 17:14:03.789074: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (penguin): /proc/driver/nvidia/version does not exist
2023-01-03 17:14:04.091676: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/10
102/102 [==============================] - 16s 108ms/step - loss: 0.3947 - auc: 0.5693 - val_loss: 0.2418 - val_auc: 0.7688
Epoch 2/10
102/102 [==============================] - 9s 85ms/step - loss: 0.2367 - auc: 0.7946 - val_loss: 0.2204 - val_auc: 0.8352
Epoch 3/10
102/102 [==============================] - 11s 108ms/step - loss: 0.2200 - auc: 0.8309 - val_loss: 0.2212 - val_auc: 0.8433
Epoch 4/10
102/102 [==============================] - 13s 128ms/step - loss: 0.2127 - auc: 0.8483 - val_loss: 0.2021 - val_auc: 0.8518
Epoch 5/10
102/102 [==============================] - 9s 86ms/step - loss: 0.2070 - auc: 0.8599 - val_loss: 0.2031 - val_auc: 0.8564
Epoch 6/10
102/102 [==============================] - 10s 98ms/step - loss: 0.2012 - auc: 0.8720 - val_loss: 0.2045 - val_auc: 0.8697
Epoch 7/10
102/102 [==============================] - 10s 95ms/step - loss: 0.1952 - auc: 0.8852 - val_loss: 0.2008 - val_auc: 0.8723
Epoch 8/10
102/102 [==============================] - 9s 84ms

[0.16040872037410736, 0.9069408178329468]

# Modified Transformer Example

In [9]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalization and Attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout)(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res

In [18]:
def build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    lstm_units,
    dropout=0,
    mlp_dropout=0,
):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    x = layers.Masking(mask_value=-99.,input_shape= input_shape)(x)
    x = layers.LSTM(lstm_units, input_shape = input_shape, return_sequences=True)(x)

    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_last")(x)
    x = layers.Dropout(0.1)(x)
    
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    
    outputs = layers.Dense(y.shape[1], activation="sigmoid")(x)
    return keras.Model(inputs, outputs)

In [15]:
input_shape = X_train.shape[1:]

model = build_model(
    input_shape,
    head_size=64,
    num_heads=4,
    ff_dim=32,
    num_transformer_blocks=8,
    mlp_units=[64],
    mlp_dropout=0.2,
    dropout=0.1,
    
)

model.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    #optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    metrics = [tf.keras.metrics.AUC()]
)
#model.summary()

callbacks = [keras.callbacks.EarlyStopping(patience=10, \
    restore_best_weights=True)]

model.fit(
    X_train,
    y_train,
    validation_split=0.15,
    epochs=NUM_EPOCHS,
    #batch_size=64,
    callbacks=callbacks,
)

model.evaluate(X_test, y_test, verbose=1)

Epoch 1/10
102/102 [==============================] - 107s 826ms/step - loss: 0.3208 - auc_1: 0.5175 - val_loss: 0.2355 - val_auc_1: 0.7411
Epoch 2/10
102/102 [==============================] - 83s 818ms/step - loss: 0.2544 - auc_1: 0.7471 - val_loss: 0.2508 - val_auc_1: 0.8218
Epoch 3/10
102/102 [==============================] - 84s 827ms/step - loss: 0.2213 - auc_1: 0.8342 - val_loss: 0.1983 - val_auc_1: 0.8801
Epoch 4/10
102/102 [==============================] - 83s 810ms/step - loss: 0.2090 - auc_1: 0.8628 - val_loss: 0.1821 - val_auc_1: 0.8943
Epoch 5/10
102/102 [==============================] - 87s 855ms/step - loss: 0.1912 - auc_1: 0.8883 - val_loss: 0.1951 - val_auc_1: 0.9010
Epoch 6/10
102/102 [==============================] - 91s 892ms/step - loss: 0.1848 - auc_1: 0.9013 - val_loss: 0.2036 - val_auc_1: 0.8972
Epoch 7/10
102/102 [==============================] - 93s 911ms/step - loss: 0.1839 - auc_1: 0.8998 - val_loss: 0.1945 - val_auc_1: 0.8814
Epoch 8/10
102/102 [======

[0.14161059260368347, 0.9273214340209961]

In [16]:
model = build_model(
    input_shape,
    head_size=64,
    num_heads=4,
    ff_dim=32,
    num_transformer_blocks=4,
    mlp_units=[32],
    mlp_dropout=0.2,
    dropout=0.2,
)

model.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    #optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    metrics = [tf.keras.metrics.AUC()]
)
#model.summary()

callbacks = [keras.callbacks.EarlyStopping(patience=10, \
    restore_best_weights=True)]

model.fit(
    X_train,
    y_train,
    validation_split=0.15,
    epochs=NUM_EPOCHS,
    #batch_size=64,
    callbacks=callbacks,
)

model.evaluate(X_test, y_test, verbose=1)

Epoch 1/10
102/102 [==============================] - 59s 457ms/step - loss: 0.2956 - auc_2: 0.6103 - val_loss: 0.2804 - val_auc_2: 0.7631
Epoch 2/10
102/102 [==============================] - 45s 445ms/step - loss: 0.2638 - auc_2: 0.7165 - val_loss: 0.2290 - val_auc_2: 0.8386
Epoch 3/10
102/102 [==============================] - 46s 449ms/step - loss: 0.2251 - auc_2: 0.8332 - val_loss: 0.1888 - val_auc_2: 0.8784
Epoch 4/10
102/102 [==============================] - 46s 455ms/step - loss: 0.1971 - auc_2: 0.8769 - val_loss: 0.1840 - val_auc_2: 0.8820
Epoch 5/10
102/102 [==============================] - 46s 449ms/step - loss: 0.2041 - auc_2: 0.8680 - val_loss: 0.1936 - val_auc_2: 0.8875
Epoch 6/10
102/102 [==============================] - 46s 451ms/step - loss: 0.1904 - auc_2: 0.8898 - val_loss: 0.2076 - val_auc_2: 0.8790
Epoch 7/10
102/102 [==============================] - 48s 471ms/step - loss: 0.1880 - auc_2: 0.8937 - val_loss: 0.1861 - val_auc_2: 0.8938
Epoch 8/10
102/102 [=======

[0.14775151014328003, 0.9158893823623657]

In [20]:
model = build_model(
    input_shape,
    head_size=64,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=2,
    mlp_units=[32],
    mlp_dropout=0.2,
    dropout=0.2,
    lstm_units=32
)

model.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    #optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    metrics = [tf.keras.metrics.AUC()]
)
#model.summary()

callbacks = [keras.callbacks.EarlyStopping(patience=10, \
    restore_best_weights=True)]

model.fit(
    X_train,
    y_train,
    validation_split=0.15,
    epochs=NUM_EPOCHS,
    #batch_size=64,
    callbacks=callbacks,
)

model.evaluate(X_test, y_test, verbose=1)

Epoch 1/10
102/102 [==============================] - 37s 281ms/step - loss: 0.2783 - auc_4: 0.6410 - val_loss: 0.2561 - val_auc_4: 0.7875
Epoch 2/10
102/102 [==============================] - 27s 261ms/step - loss: 0.2400 - auc_4: 0.7796 - val_loss: 0.2303 - val_auc_4: 0.8531
Epoch 3/10
102/102 [==============================] - 27s 267ms/step - loss: 0.2097 - auc_4: 0.8570 - val_loss: 0.1871 - val_auc_4: 0.8821
Epoch 4/10
102/102 [==============================] - 27s 267ms/step - loss: 0.1965 - auc_4: 0.8801 - val_loss: 0.1842 - val_auc_4: 0.8823
Epoch 5/10
102/102 [==============================] - 27s 266ms/step - loss: 0.1983 - auc_4: 0.8794 - val_loss: 0.1992 - val_auc_4: 0.8778
Epoch 6/10
102/102 [==============================] - 27s 263ms/step - loss: 0.1842 - auc_4: 0.9008 - val_loss: 0.2054 - val_auc_4: 0.8757
Epoch 7/10
102/102 [==============================] - 27s 265ms/step - loss: 0.1829 - auc_4: 0.9018 - val_loss: 0.1910 - val_auc_4: 0.8808
Epoch 8/10
102/102 [=======

[0.1453460156917572, 0.9244112968444824]

In [22]:
model = build_model(
    input_shape,
    head_size=128,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=1,
    mlp_units=[64],
    mlp_dropout=0.2,
    dropout=0.2,
    lstm_units=32
)

model.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    #optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    metrics = [tf.keras.metrics.AUC()]
)
#model.summary()

callbacks = [keras.callbacks.EarlyStopping(patience=10, \
    restore_best_weights=True)]

model.fit(
    X_train,
    y_train,
    validation_split=0.15,
    epochs=NUM_EPOCHS,
    #batch_size=64,
    callbacks=callbacks,
)

model.evaluate(X_test, y_test, verbose=1)

Epoch 1/10
102/102 [==============================] - 30s 234ms/step - loss: 0.2757 - auc_6: 0.6464 - val_loss: 0.2459 - val_auc_6: 0.7897
Epoch 2/10
102/102 [==============================] - 21s 209ms/step - loss: 0.2297 - auc_6: 0.8012 - val_loss: 0.2065 - val_auc_6: 0.8761
Epoch 3/10
102/102 [==============================] - 22s 215ms/step - loss: 0.2017 - auc_6: 0.8708 - val_loss: 0.1864 - val_auc_6: 0.8865
Epoch 4/10
102/102 [==============================] - 23s 227ms/step - loss: 0.1953 - auc_6: 0.8836 - val_loss: 0.1905 - val_auc_6: 0.8746
Epoch 5/10
102/102 [==============================] - 23s 221ms/step - loss: 0.1889 - auc_6: 0.8897 - val_loss: 0.1856 - val_auc_6: 0.8827
Epoch 6/10
102/102 [==============================] - 22s 218ms/step - loss: 0.1750 - auc_6: 0.9128 - val_loss: 0.1976 - val_auc_6: 0.8949
Epoch 7/10
102/102 [==============================] - 23s 225ms/step - loss: 0.1713 - auc_6: 0.9177 - val_loss: 0.1906 - val_auc_6: 0.8891
Epoch 8/10
102/102 [=======

[0.1477433741092682, 0.922060489654541]

In [23]:
model = build_model(
    input_shape,
    head_size=128,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=1,
    mlp_units=[128],
    mlp_dropout=0.2,
    dropout=0.4,
    lstm_units=32
)

model.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    #optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    metrics = [tf.keras.metrics.AUC()]
)
#model.summary()

callbacks = [keras.callbacks.EarlyStopping(min_delta=0.01, patience=3, restore_best_weights=True)]

model.fit(
    X_train,
    y_train,
    validation_split=0.15,
    epochs=NUM_EPOCHS,
    #batch_size=64,
    callbacks=callbacks,
)

model.evaluate(X_test, y_test, verbose=1)

Epoch 1/10
102/102 [==============================] - 30s 235ms/step - loss: 0.2813 - auc_7: 0.6264 - val_loss: 0.2547 - val_auc_7: 0.7755
Epoch 2/10
102/102 [==============================] - 22s 211ms/step - loss: 0.2395 - auc_7: 0.7681 - val_loss: 0.1984 - val_auc_7: 0.8617
Epoch 3/10
102/102 [==============================] - 22s 218ms/step - loss: 0.2183 - auc_7: 0.8290 - val_loss: 0.1951 - val_auc_7: 0.8771
Epoch 4/10
102/102 [==============================] - 23s 222ms/step - loss: 0.2150 - auc_7: 0.8477 - val_loss: 0.1906 - val_auc_7: 0.8814
Epoch 5/10
102/102 [==============================] - 23s 221ms/step - loss: 0.2045 - auc_7: 0.8663 - val_loss: 0.1919 - val_auc_7: 0.8705
Epoch 6/10
102/102 [==============================] - 23s 225ms/step - loss: 0.1909 - auc_7: 0.8896 - val_loss: 0.1756 - val_auc_7: 0.9037
Epoch 7/10
102/102 [==============================] - 25s 244ms/step - loss: 0.1868 - auc_7: 0.8965 - val_loss: 0.1815 - val_auc_7: 0.8883
Epoch 8/10
102/102 [=======

[0.1385122388601303, 0.9286485314369202]

In [24]:
model = build_model(
    input_shape,
    head_size=128,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=1,
    mlp_units=[128],
    mlp_dropout=0.2,
    dropout=0.25,
    lstm_units=32
)

model.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    #optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    metrics = [tf.keras.metrics.AUC()]
)
#model.summary()

callbacks = [keras.callbacks.EarlyStopping(min_delta=0.01, patience=3, restore_best_weights=True)]

model.fit(
    X_train,
    y_train,
    validation_split=0.15,
    epochs=NUM_EPOCHS,
    #batch_size=64,
    callbacks=callbacks,
)

model.evaluate(X_test, y_test, verbose=1)

Epoch 1/10
102/102 [==============================] - 36s 242ms/step - loss: 0.2759 - auc_8: 0.6545 - val_loss: 0.2334 - val_auc_8: 0.8179
Epoch 2/10
102/102 [==============================] - 22s 216ms/step - loss: 0.2262 - auc_8: 0.8084 - val_loss: 0.2067 - val_auc_8: 0.8731
Epoch 3/10
102/102 [==============================] - 23s 224ms/step - loss: 0.2040 - auc_8: 0.8666 - val_loss: 0.1905 - val_auc_8: 0.8834
Epoch 4/10
102/102 [==============================] - 23s 224ms/step - loss: 0.2007 - auc_8: 0.8746 - val_loss: 0.1896 - val_auc_8: 0.8775
Epoch 5/10
102/102 [==============================] - 25s 249ms/step - loss: 0.1901 - auc_8: 0.8892 - val_loss: 0.1849 - val_auc_8: 0.8851
Epoch 6/10
102/102 [==============================] - 24s 238ms/step - loss: 0.1757 - auc_8: 0.9115 - val_loss: 0.1717 - val_auc_8: 0.9095
Epoch 7/10
102/102 [==============================] - 26s 252ms/step - loss: 0.1745 - auc_8: 0.9121 - val_loss: 0.1812 - val_auc_8: 0.8888
Epoch 8/10
102/102 [=======

[0.13807593286037445, 0.9324971437454224]